# Sistema de clasificación

In [12]:
# from modules import *
import json
import plotly.graph_objects as go
import numpy as np
import os
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [13]:
pathdata0 = os.getcwd()+"/../data/lineas_completas/VDI711.json"
pathdata1 = os.getcwd()+"/../data/lineas_completas/REG804.json"
pathdata2 = os.getcwd()+"/../data/lineas_completas/XIN803.json"

In [14]:
def get_coord(points):
    
    x_vals = [punto[0] for punto in points]
    y_vals = [punto[1] for punto in points]
    z_vals = [punto[2] for punto in points]
    
    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

def get_coord2(extremos_apoyos):
    
    x_vals = [extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"]]
    y_vals = [extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"]]
    z_vals = extremos_apoyos[0]["COORDENADAS_Z"] + extremos_apoyos[1]["COORDENADAS_Z"]

    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

def unravel_data_element(element):
    
    for key in element.keys():
    
        if type(element[key]) in [list, dict]:
            
            print(f"\n{key}: ")
            
            if type(element[key]) == list:
            
                element2 = element[key][0]
                print(f"- Length of list: {len(element[key])}")
                
            else:
                
                element2 = element[key]
            
            for key2 in element2.keys():
                
                print(f"    {key2}: {element2[key2]}")
                
                if type(element2[key2]) == list:
                    print(f"    - Length of list: {len(element2[key2])}")
        
        else:
            print(f"\n{key}: {element[key]}")

In [15]:
def rotate_points(points, extremos_values):
    
    points = np.array(points).T

    extremo1 = np.array(extremos_values).T[0]  # Extremo superior del primer poste
    extremo2 = np.array(extremos_values).T[2]  # Extremo inferior del primer poste
    
    # Calcular la distancia en el plano XY y la dirección de la diagonal
    distancia_xy = np.linalg.norm(extremo2[:2] - extremo1[:2])
    direccion_diagonal = (extremo2[:2] - extremo1[:2]) / distancia_xy # Normalizada para la distancia
    
    # Calcular el ángulo de rotación necesario para alinear la diagonal con el eje Y
    angulo = np.arctan2(direccion_diagonal[1], direccion_diagonal[0])
    
    # Ajustar el ángulo para la rotación correcta
    angulo += np.pi / 2
    c, s = np.cos(angulo), np.sin(angulo)
    
    # Crear la matriz de rotación para alinear la diagonal con el eje Y
    matriz_rotacion = np.array([[c, s, 0],
                                [-s, c, 0],
                                [0, 0, 1]])
    
    rotated_points = matriz_rotacion.dot(points.T)
    # print(rotated.shape)
    
    return matriz_rotacion, np.array(rotated_points)

Para ver el ajuste de un vano

In [16]:
def ajuste_graf(data, vano):

    puntos_conductores = data[vano]['LIDAR']['CONDUCTORES']
    puntos_vertices = data[vano]['CONDUCTORES'][0]['VERTICES']
    puntos_vertices2 = data[vano]['CONDUCTORES'][1]['VERTICES']
    puntos_vertices3 = data[vano]['CONDUCTORES'][2]['VERTICES']
    puntos_extremos = data[vano]['APOYOS']

    x_vals_conductores, y_vals_conductores, z_vals_conductores = get_coord(puntos_conductores)
    x_vals_extremos, y_vals_extremos, z_vals_extremos = get_coord2(puntos_extremos)
    x_vert1, y_vert1, z_vert1 = get_coord(puntos_vertices)
    x_vert2, y_vert2, z_vert2 = get_coord(puntos_vertices2)
    x_vert3, y_vert3, z_vert3 = get_coord(puntos_vertices3)

    cond_values = [x_vals_conductores, y_vals_conductores, z_vals_conductores]
    extremos_values = [x_vals_extremos, y_vals_extremos, z_vals_extremos]
    vert_values1 = [x_vert1, y_vert1, z_vert1]
    vert_values2 = [x_vert2, y_vert2, z_vert2]
    vert_values3 = [x_vert3, y_vert3, z_vert3]

    # Matriz de rotación
    mat, rotated_conds = rotate_points(cond_values, extremos_values)
    extremos_values = mat.dot(extremos_values)
    rotated_vertices1 = mat.dot(vert_values1)
    rotated_vertices2 = mat.dot(vert_values2)
    rotated_vertices3 = mat.dot(vert_values3)

    X_extremos = extremos_values[0]
    Y_extremos = extremos_values[1]
    Z_extremos = extremos_values[2]

    X_cond = rotated_conds[0]
    Y_cond = rotated_conds[1]
    Z_cond = rotated_conds[2]

    # Filtramos los puntos de los conductores que están entre los extremos
    x = []
    y = []
    z = []

    for i in range(len(X_cond)):
        if Y_cond[i] > np.min(Y_extremos) and Y_cond[i] < np.max(Y_extremos):
            x.append(X_cond[i])
            y.append(Y_cond[i])
            z.append(Z_cond[i])

    x_cond = np.array(x)
    y_cond = np.array(y)
    z_cond = np.array(z)

    # Clustering
    [X, y] = [x_cond.reshape(-1, 1), y_cond.reshape(-1, 1)]

    model = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=0)

    y_spectral = model.fit_predict(X)

    # Separamos los 3 cables
    x1, x2, x3 = [], [], []
    y1, y2, y3 = [], [], []
    z1, z2, z3 = [], [], []

    for i in range(0, len(y_spectral)):
        if y_spectral[i] == 0:
            x1.append(X[i])
            y1.append(y[i])
            z1.append(z[i])
        if y_spectral[i] == 1:
            x2.append(X[i])
            y2.append(y[i])
            z2.append(z[i])
        if y_spectral[i] == 2:
            x3.append(X[i])
            y3.append(y[i])
            z3.append(z[i])

    x1, y1, z1 = np.array(x1), np.array(y1), np.array(z1)
    x2, y2, z2 = np.array(x2), np.array(y2), np.array(z2)
    x3, y3, z3 = np.array(x3), np.array(y3), np.array(z3)

    # Ajuste de la catenaria
    data_2d_cond1 = np.column_stack((y1, z1))
    data_2d_cond2 = np.column_stack((y2, z2))
    data_2d_cond3 = np.column_stack((y3, z3))

    pca = PCA(n_components=2)
    data_2d_pca_cond1 = pca.fit_transform(data_2d_cond1)
    data_2d_pca_cond2 = pca.fit_transform(data_2d_cond2)
    data_2d_pca_cond3 = pca.fit_transform(data_2d_cond3)

    y_min_cond1, y_max_cond1 = data_2d_pca_cond1[:, 1].min(), data_2d_pca_cond1[:,1].max()
    y_min_cond2, y_max_cond2 = data_2d_pca_cond2[:, 1].min(), data_2d_pca_cond2[:,1].max()
    y_min_cond3, y_max_cond3 = data_2d_pca_cond3[:, 1].min(), data_2d_pca_cond3[:,1].max()

    f_ind1 = (data_2d_pca_cond1[:,1] > y_min_cond1) & (data_2d_pca_cond1[:,1] < y_max_cond1)
    f_ind2 = (data_2d_pca_cond2[:,1] > y_min_cond2) & (data_2d_pca_cond2[:,1] < y_max_cond2)
    f_ind3 = (data_2d_pca_cond3[:,1] > y_min_cond3) & (data_2d_pca_cond3[:,1] < y_max_cond3)
    x_filt_cond1, y_filt_cond1, z_filt_cond1 = x1[f_ind1], y1[f_ind1], z1[f_ind1]
    x_filt_cond2, y_filt_cond2, z_filt_cond2 = x2[f_ind2], y2[f_ind2], z2[f_ind2]
    x_filt_cond3, y_filt_cond3, z_filt_cond3 = x3[f_ind3], y3[f_ind3], z3[f_ind3]

    # Función de la catenaria
    from sklearn.preprocessing import StandardScaler
    from scipy.optimize import curve_fit
    def catenaria(x, a, h, k):
        return a*np.cosh((x-h)/a)+k

    y_vals1 = y_filt_cond1.reshape(-1, 1)
    z_vals1 = z_filt_cond1.reshape(-1, 1)
    y_vals2 = y_filt_cond2.reshape(-1, 1)
    z_vals2 = z_filt_cond2.reshape(-1, 1)
    y_vals3 = y_filt_cond3.reshape(-1, 1)
    z_vals3 = z_filt_cond3.reshape(-1, 1)


    scaler_y1 = StandardScaler()
    scaler_z1 = StandardScaler()
    scaler_y2 = StandardScaler()
    scaler_z2 = StandardScaler()
    scaler_y3 = StandardScaler()
    scaler_z3 = StandardScaler()

    y_vals_scaled1 = scaler_y1.fit_transform(y_vals1).flatten()
    z_vals_scaled1 = scaler_z1.fit_transform(z_vals1).flatten()
    y_vals_scaled2 = scaler_y2.fit_transform(y_vals2).flatten()
    z_vals_scaled2 = scaler_z2.fit_transform(z_vals2).flatten()
    y_vals_scaled3 = scaler_y3.fit_transform(y_vals3).flatten()
    z_vals_scaled3 = scaler_z3.fit_transform(z_vals3).flatten()

    p0 = [1, 0, 0]

    parametros1, _ = curve_fit(catenaria, y_vals_scaled1.flatten(), z_vals_scaled1)
    parametros2, _ = curve_fit(catenaria, y_vals_scaled2.flatten(), z_vals_scaled2)
    parametros3, _ = curve_fit(catenaria, y_vals_scaled3.flatten(), z_vals_scaled3)

    # Ajuste de los puntos de los datos a una catenaria
    fitted_z_vals_scaled1 = catenaria(y_vals_scaled1.flatten(), *parametros1)
    fitted_z_vals1 = scaler_z1.inverse_transform(fitted_z_vals_scaled1.reshape(-1, 1)).flatten()
    fitted_z_vals_scaled2 = catenaria(y_vals_scaled2.flatten(), *parametros2)
    fitted_z_vals2 = scaler_z2.inverse_transform(fitted_z_vals_scaled2.reshape(-1, 1)).flatten()
    fitted_z_vals_scaled3 = catenaria(y_vals_scaled3.flatten(), *parametros3)
    fitted_z_vals3 = scaler_z3.inverse_transform(fitted_z_vals_scaled3.reshape(-1, 1)).flatten()

    plt.figure(figsize=(10, 6))
    # Pintamos los puntos de cada cable
    plt.scatter(y1, z1, color='coral', s=30)
    plt.scatter(y2, z2, color='lightblue', s=30)
    plt.scatter(y3, z3, color='lightgreen', s=30)

    # Interpolación de la polilínea
    minimo1 = np.min(scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten())
    maximo1 = np.max(scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten())
    x_pol1 = np.linspace(minimo1, maximo1, 1000)

    minimo2 = np.min(scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten())
    maximo2 = np.max(scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten())
    x_pol2 = np.linspace(minimo2, maximo2, 1000)

    minimo3 = np.min(scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten())
    maximo3 = np.max(scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten())
    x_pol3 = np.linspace(minimo3, maximo3, 1000)

    scaler_x1 = StandardScaler()
    scaler_x2 = StandardScaler()
    scaler_x3 = StandardScaler()

    x_scaled1 = scaler_x1.fit_transform(x_pol1.reshape(-1, 1)).flatten()
    x_scaled2 = scaler_x2.fit_transform(x_pol2.reshape(-1, 1)).flatten()
    x_scaled3 = scaler_x3.fit_transform(x_pol3.reshape(-1, 1)).flatten()

    fitted_y_scaled1 = catenaria(x_scaled1.flatten(), *parametros1)
    fitted_y1 = scaler_z1.inverse_transform(fitted_y_scaled1.reshape(-1, 1)).flatten()
    fitted_y_scaled2 = catenaria(x_scaled2.flatten(), *parametros2)
    fitted_y2 = scaler_z2.inverse_transform(fitted_y_scaled2.reshape(-1, 1)).flatten()
    fitted_y_scaled3 = catenaria(x_scaled3.flatten(), *parametros3)
    fitted_y3 = scaler_z3.inverse_transform(fitted_y_scaled3.reshape(-1, 1)).flatten()

    y_pol1 = np.interp(x_pol1, scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten(), fitted_z_vals1, period=1000)
    # y_pol2 = np.interp(x_pol2, scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten(), fitted_z_vals2, period=len(fitted_z_vals3))
    y_pol2 = np.interp(x_pol2, scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten(), fitted_z_vals2, period=1000)
    y_pol3 = np.interp(x_pol3, scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten(), fitted_z_vals3, period=1000)

    # Pintamos las polilíneas que hemos generado
    plt.plot(x_pol1, y_pol1, color='red', label='P1')
    plt.plot(x_pol2, y_pol2, color='blue', label='P2')
    plt.plot(x_pol3, y_pol3, color='green', label='P3')

    # Pintamos las polilíneas que nos dan con los datos
    plt.scatter(rotated_vertices1[1], rotated_vertices1[2], color='red', label='Polilínea 1', s=30)
    plt.scatter(rotated_vertices2[1], rotated_vertices2[2], color='blue', label='Polilínea 2', s=30)
    plt.scatter(rotated_vertices3[1], rotated_vertices3[2], color='green', label='Polilínea 3', s=30)

    plt.legend()
    plt.title(vano)

Para obtener las polilíneas del ajuste

In [17]:
def ajuste(data, vano):

    puntos_conductores = data[vano]['LIDAR']['CONDUCTORES']
    puntos_vertices = data[vano]['CONDUCTORES'][0]['VERTICES']
    puntos_vertices2 = data[vano]['CONDUCTORES'][1]['VERTICES']
    puntos_vertices3 = data[vano]['CONDUCTORES'][2]['VERTICES']
    puntos_extremos = data[vano]['APOYOS']

    x_vals_conductores, y_vals_conductores, z_vals_conductores = get_coord(puntos_conductores)
    x_vals_extremos, y_vals_extremos, z_vals_extremos = get_coord2(puntos_extremos)
    x_vert1, y_vert1, z_vert1 = get_coord(puntos_vertices)
    x_vert2, y_vert2, z_vert2 = get_coord(puntos_vertices2)
    x_vert3, y_vert3, z_vert3 = get_coord(puntos_vertices3)

    cond_values = [x_vals_conductores, y_vals_conductores, z_vals_conductores]
    extremos_values = [x_vals_extremos, y_vals_extremos, z_vals_extremos]
    vert_values1 = [x_vert1, y_vert1, z_vert1]
    vert_values2 = [x_vert2, y_vert2, z_vert2]
    vert_values3 = [x_vert3, y_vert3, z_vert3]

    # Matriz de rotación
    mat, rotated_conds = rotate_points(cond_values, extremos_values)
    extremos_values = mat.dot(extremos_values)
    rotated_vertices1 = mat.dot(vert_values1)
    rotated_vertices2 = mat.dot(vert_values2)
    rotated_vertices3 = mat.dot(vert_values3)

    X_extremos = extremos_values[0]
    Y_extremos = extremos_values[1]
    Z_extremos = extremos_values[2]

    X_cond = rotated_conds[0]
    Y_cond = rotated_conds[1]
    Z_cond = rotated_conds[2]

    # Filtramos los puntos de los conductores que están entre los extremos
    x = []
    y = []
    z = []

    for i in range(len(X_cond)):
        if Y_cond[i] > np.min(Y_extremos) and Y_cond[i] < np.max(Y_extremos):
            x.append(X_cond[i])
            y.append(Y_cond[i])
            z.append(Z_cond[i])

    x_cond = np.array(x)
    y_cond = np.array(y)
    z_cond = np.array(z)

    # Clustering
    [X, y] = [x_cond.reshape(-1, 1), y_cond.reshape(-1, 1)]

    model = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=0)

    y_spectral = model.fit_predict(X)

    # Separamos los 3 cables
    x1, x2, x3 = [], [], []
    y1, y2, y3 = [], [], []
    z1, z2, z3 = [], [], []

    for i in range(0, len(y_spectral)):
        if y_spectral[i] == 0:
            x1.append(X[i])
            y1.append(y[i])
            z1.append(z[i])
        if y_spectral[i] == 1:
            x2.append(X[i])
            y2.append(y[i])
            z2.append(z[i])
        if y_spectral[i] == 2:
            x3.append(X[i])
            y3.append(y[i])
            z3.append(z[i])

    x1, y1, z1 = np.array(x1), np.array(y1), np.array(z1)
    x2, y2, z2 = np.array(x2), np.array(y2), np.array(z2)
    x3, y3, z3 = np.array(x3), np.array(y3), np.array(z3)

    # Ajuste de la catenaria
    data_2d_cond1 = np.column_stack((y1, z1))
    data_2d_cond2 = np.column_stack((y2, z2))
    data_2d_cond3 = np.column_stack((y3, z3))

    pca = PCA(n_components=2)
    data_2d_pca_cond1 = pca.fit_transform(data_2d_cond1)
    data_2d_pca_cond2 = pca.fit_transform(data_2d_cond2)
    data_2d_pca_cond3 = pca.fit_transform(data_2d_cond3)

    y_min_cond1, y_max_cond1 = data_2d_pca_cond1[:, 1].min(), data_2d_pca_cond1[:,1].max()
    y_min_cond2, y_max_cond2 = data_2d_pca_cond2[:, 1].min(), data_2d_pca_cond2[:,1].max()
    y_min_cond3, y_max_cond3 = data_2d_pca_cond3[:, 1].min(), data_2d_pca_cond3[:,1].max()

    f_ind1 = (data_2d_pca_cond1[:,1] > y_min_cond1) & (data_2d_pca_cond1[:,1] < y_max_cond1)
    f_ind2 = (data_2d_pca_cond2[:,1] > y_min_cond2) & (data_2d_pca_cond2[:,1] < y_max_cond2)
    f_ind3 = (data_2d_pca_cond3[:,1] > y_min_cond3) & (data_2d_pca_cond3[:,1] < y_max_cond3)
    x_filt_cond1, y_filt_cond1, z_filt_cond1 = x1[f_ind1], y1[f_ind1], z1[f_ind1]
    x_filt_cond2, y_filt_cond2, z_filt_cond2 = x2[f_ind2], y2[f_ind2], z2[f_ind2]
    x_filt_cond3, y_filt_cond3, z_filt_cond3 = x3[f_ind3], y3[f_ind3], z3[f_ind3]

    # Función de la catenaria
    from sklearn.preprocessing import StandardScaler
    from scipy.optimize import curve_fit
    def catenaria(x, a, h, k):
        return a*np.cosh((x-h)/a)+k

    y_vals1 = y_filt_cond1.reshape(-1, 1)
    z_vals1 = z_filt_cond1.reshape(-1, 1)
    y_vals2 = y_filt_cond2.reshape(-1, 1)
    z_vals2 = z_filt_cond2.reshape(-1, 1)
    y_vals3 = y_filt_cond3.reshape(-1, 1)
    z_vals3 = z_filt_cond3.reshape(-1, 1)


    scaler_y1 = StandardScaler()
    scaler_z1 = StandardScaler()
    scaler_y2 = StandardScaler()
    scaler_z2 = StandardScaler()
    scaler_y3 = StandardScaler()
    scaler_z3 = StandardScaler()

    y_vals_scaled1 = scaler_y1.fit_transform(y_vals1).flatten()
    z_vals_scaled1 = scaler_z1.fit_transform(z_vals1).flatten()
    y_vals_scaled2 = scaler_y2.fit_transform(y_vals2).flatten()
    z_vals_scaled2 = scaler_z2.fit_transform(z_vals2).flatten()
    y_vals_scaled3 = scaler_y3.fit_transform(y_vals3).flatten()
    z_vals_scaled3 = scaler_z3.fit_transform(z_vals3).flatten()

    p0 = [1, 0, 0]

    parametros1, _ = curve_fit(catenaria, y_vals_scaled1.flatten(), z_vals_scaled1)
    parametros2, _ = curve_fit(catenaria, y_vals_scaled2.flatten(), z_vals_scaled2)
    parametros3, _ = curve_fit(catenaria, y_vals_scaled3.flatten(), z_vals_scaled3)

    # Ajuste de los puntos de los datos a una catenaria
    fitted_z_vals_scaled1 = catenaria(y_vals_scaled1.flatten(), *parametros1)
    fitted_z_vals1 = scaler_z1.inverse_transform(fitted_z_vals_scaled1.reshape(-1, 1)).flatten()
    fitted_z_vals_scaled2 = catenaria(y_vals_scaled2.flatten(), *parametros2)
    fitted_z_vals2 = scaler_z2.inverse_transform(fitted_z_vals_scaled2.reshape(-1, 1)).flatten()
    fitted_z_vals_scaled3 = catenaria(y_vals_scaled3.flatten(), *parametros3)
    fitted_z_vals3 = scaler_z3.inverse_transform(fitted_z_vals_scaled3.reshape(-1, 1)).flatten()

    # Interpolación de la polilínea
    minimo1 = np.min(scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten())
    maximo1 = np.max(scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten())
    x_pol1 = np.linspace(minimo1, maximo1, 1000)

    minimo2 = np.min(scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten())
    maximo2 = np.max(scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten())
    x_pol2 = np.linspace(minimo2, maximo2, 1000)

    minimo3 = np.min(scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten())
    maximo3 = np.max(scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten())
    x_pol3 = np.linspace(minimo3, maximo3, 1000)

    scaler_x1 = StandardScaler()
    scaler_x2 = StandardScaler()
    scaler_x3 = StandardScaler()

    x_scaled1 = scaler_x1.fit_transform(x_pol1.reshape(-1, 1)).flatten()
    x_scaled2 = scaler_x2.fit_transform(x_pol2.reshape(-1, 1)).flatten()
    x_scaled3 = scaler_x3.fit_transform(x_pol3.reshape(-1, 1)).flatten()

    fitted_y_scaled1 = catenaria(x_scaled1.flatten(), *parametros1)
    fitted_y1 = scaler_z1.inverse_transform(fitted_y_scaled1.reshape(-1, 1)).flatten()
    fitted_y_scaled2 = catenaria(x_scaled2.flatten(), *parametros2)
    fitted_y2 = scaler_z2.inverse_transform(fitted_y_scaled2.reshape(-1, 1)).flatten()
    fitted_y_scaled3 = catenaria(x_scaled3.flatten(), *parametros3)
    fitted_y3 = scaler_z3.inverse_transform(fitted_y_scaled3.reshape(-1, 1)).flatten()

    y_pol1 = np.interp(x_pol1, scaler_y1.inverse_transform(y_vals_scaled1.reshape(-1, 1)).flatten(), fitted_z_vals1, period=1000)
    y_pol2 = np.interp(x_pol2, scaler_y2.inverse_transform(y_vals_scaled2.reshape(-1, 1)).flatten(), fitted_z_vals2, period=1000)
    y_pol3 = np.interp(x_pol3, scaler_y3.inverse_transform(y_vals_scaled3.reshape(-1, 1)).flatten(), fitted_z_vals3, period=1000)

    return(x_pol1, y_pol1, x_pol2, y_pol2, x_pol3, y_pol3)

Error entre nuestra polilínea (x) y la polilínea dada (y) que siempre va a ser más pequeña

Sacamos el mismo número de puntos que la longitud de la polilínea dada, separados con un intervalo regular, y calculamos el rmse entre las dos líneas

In [18]:
def rmse(x, y):
        intervalo = len(x) // (len(y)-1)
        nn = [x[i * intervalo] for i in range(len(y)-1)] + [x[-1]]
        return np.sqrt(mean_squared_error(nn, y))

Sistema de puntuación de los datos que nos dan

In [19]:
def puntuación(pathdata0):
    with open(pathdata0, 'r') as archivo:
        data = json.load(archivo)

    clasificacion1 = {
        'Vano': [],
        'Apoyos': [],
        'Conductores': [],
        'Longitud 2D': [],
        'Longitud polilínea 1': [],
        'Longitud polilínea 2': [],
        'Longitud polilínea 3': [],
        'Puntuación': [],
    }
    for iel, el in enumerate(data):
        apoyos = el['APOYOS']
        conductores = el['CONDUCTORES']
        len_apoyos = len(apoyos)
        len_conductores = len(conductores)
        clasificacion1['Vano'].append(el['ID_VANO'])
        clasificacion1['Apoyos'].append(len_apoyos)
        clasificacion1['Conductores'].append(len_conductores)
        longitud = el['LONGITUD_2D']
        clasificacion1['Longitud 2D'].append(longitud)
        nota = 10
        if len_apoyos == 2:
            if len_conductores >= 3:
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                x2, y2, z2 = get_coord(conductores[1]['VERTICES'])
                x3, y3, z3 = get_coord(conductores[2]['VERTICES'])
                x_cond, y_cond, z_cond = get_coord(el['LIDAR']['CONDUCTORES'])
                x_extremos, y_extremos, z_extremos = get_coord2(apoyos)
                cond_values = [x_cond, y_cond, z_cond]
                extremos_values = [x_extremos, y_extremos, z_extremos]
                mat, rotated_conds = rotate_points(cond_values, extremos_values)
                vert1 = [x1, y1, z1]
                vert2 = [x2, y2, z2]
                vert3 = [x3, y3, z3]
                rot_vert1 = mat.dot(vert1)
                rot_vert2 = mat.dot(vert2)
                rot_vert3 = mat.dot(vert3)
                x1, y1, z1 = rot_vert1[0], rot_vert1[1], rot_vert1[2]
                x2, y2, z2 = rot_vert2[0], rot_vert2[1], rot_vert2[2]
                x3, y3, z3 = rot_vert3[0], rot_vert3[1], rot_vert3[2]
                
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                long_pol2 = np.sqrt((x2[0]-x2[-1])**2 + (y2[0]-y2[-1])**2)
                long_pol3 = np.sqrt((x3[0]-x3[-1])**2 + (y3[0]-y3[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(long_pol2)
                clasificacion1['Longitud polilínea 3'].append(long_pol3)
                error = 5
                longitudes = [long_pol1, long_pol2, long_pol3]

                if long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error and long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 10
                elif long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error or long_pol1 - error <= longitud <= long_pol1 + error and long_pol3 - error <= longitud <= long_pol3 + error or long_pol2 - error <= longitud <= long_pol2 + error and long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 9
                elif long_pol1 - error <= longitud <= long_pol1 + error or long_pol2 - error <= longitud <= long_pol2 + error or long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 8
                else:
                    nota = 7
            elif len_conductores == 2:
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                x2, y2, z2 = get_coord(conductores[1]['VERTICES'])
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                long_pol2 = np.sqrt((x2[0]-x2[-1])**2 + (y2[0]-y2[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(long_pol2)
                clasificacion1['Longitud polilínea 3'].append(0)
                error = 5
                if long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error:
                    nota = 6
                elif long_pol1 - error <= longitud <= long_pol1 + error or long_pol2 - error <= longitud <= long_pol2 + error:
                    nota = 5
                else:
                    nota = 4
            elif len_conductores == 1:
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(0)
                clasificacion1['Longitud polilínea 3'].append(0)
                error = 5
                if long_pol1 - error <= longitud <= long_pol1 + error:
                    nota = 3
                else:
                    nota = 2
            else:
                clasificacion1['Longitud polilínea 1'].append(0)
                clasificacion1['Longitud polilínea 2'].append(0)
                clasificacion1['Longitud polilínea 3'].append(0)
                nota = 1
            clasificacion1['Puntuación'].append(nota)
        else:
            clasificacion1['Puntuación'].append(0)
            clasificacion1['Longitud polilínea 1'].append(0)
            clasificacion1['Longitud polilínea 2'].append(0)
            clasificacion1['Longitud polilínea 3'].append(0)

    clasificacion1 = pd.DataFrame(clasificacion1)
    return(clasificacion1)

Sistema de puntuación de la polilínea que les damos (Revisar)

In [20]:
def puntuación_pol(pathdata0):
    with open(pathdata0, 'r') as archivo:
        data = json.load(archivo)

    clasificacion1 = {
        'Vano': [],
        'Apoyos': [],
        'Conductores': [],
        'Longitud 2D': [],
        'Longitud polilínea 1': [],
        'Longitud polilínea 2': [],
        'Longitud polilínea 3': [],
        'Puntuación': [],
        'Error p1': [],
        'Error p2': [],
        'Error p3': [],
    }
    for iel, el in enumerate(data):
        apoyos = el['APOYOS']
        conductores = el['CONDUCTORES']
        len_apoyos = len(apoyos)
        len_conductores = len(conductores)
        clasificacion1['Vano'].append(el['ID_VANO'])
        clasificacion1['Apoyos'].append(len_apoyos)
        clasificacion1['Conductores'].append(len_conductores)
        longitud = el['LONGITUD_2D']
        clasificacion1['Longitud 2D'].append(longitud)
        nota = 10
        if len_apoyos == 2:
            if len_conductores >= 3:
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                x2, y2, z2 = get_coord(conductores[1]['VERTICES'])
                x3, y3, z3 = get_coord(conductores[2]['VERTICES'])
                x_cond, y_cond, z_cond = get_coord(el['LIDAR']['CONDUCTORES'])
                x_extremos, y_extremos, z_extremos = get_coord2(apoyos)
                cond_values = [x_cond, y_cond, z_cond]
                extremos_values = [x_extremos, y_extremos, z_extremos]
                mat, rotated_conds = rotate_points(cond_values, extremos_values)
                vert1 = [x1, y1, z1]
                vert2 = [x2, y2, z2]
                vert3 = [x3, y3, z3]
                rot_vert1 = mat.dot(vert1)
                rot_vert2 = mat.dot(vert2)
                rot_vert3 = mat.dot(vert3)
                x1, y1, z1 = rot_vert1[0], rot_vert1[1], rot_vert1[2]
                x2, y2, z2 = rot_vert2[0], rot_vert2[1], rot_vert2[2]
                x3, y3, z3 = rot_vert3[0], rot_vert3[1], rot_vert3[2]
                
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                long_pol2 = np.sqrt((x2[0]-x2[-1])**2 + (y2[0]-y2[-1])**2)
                long_pol3 = np.sqrt((x3[0]-x3[-1])**2 + (y3[0]-y3[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(long_pol2)
                clasificacion1['Longitud polilínea 3'].append(long_pol3)
                polx1, poly1, polx2, poly2, polx3, poly3 = ajuste(data, iel)
                errorx1 = rmse(polx1, y1)
                errory1 = rmse(poly1, z1)
                errorx2 = rmse(polx2, y2)
                errory2 = rmse(poly2, z2)
                errorx3 = rmse(polx3, y3)
                errory3 = rmse(poly3, z3)
                clasificacion1['Error p1'].append(np.sqrt(errorx1-errory1))
                clasificacion1['Error p2'].append(np.sqrt(errorx2-errory2))
                clasificacion1['Error p3'].append(np.sqrt(errorx3-errory3))
                error = 5
                longitudes = [long_pol1, long_pol2, long_pol3]

                if long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error and long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 10
                elif long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error or long_pol1 - error <= longitud <= long_pol1 + error and long_pol3 - error <= longitud <= long_pol3 + error or long_pol2 - error <= longitud <= long_pol2 + error and long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 9
                elif long_pol1 - error <= longitud <= long_pol1 + error or long_pol2 - error <= longitud <= long_pol2 + error or long_pol3 - error <= longitud <= long_pol3 + error:
                    nota = 8
                else:
                    nota = 7
            elif len_conductores == 2:
                clasificacion1['Error p1'].append(0)
                clasificacion1['Error p2'].append(0)
                clasificacion1['Error p3'].append(0)
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                x2, y2, z2 = get_coord(conductores[1]['VERTICES'])
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                long_pol2 = np.sqrt((x2[0]-x2[-1])**2 + (y2[0]-y2[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(long_pol2)
                clasificacion1['Longitud polilínea 3'].append(0)
                error = 5
                if long_pol1 - error <= longitud <= long_pol1 + error and long_pol2 - error <= longitud <= long_pol2 + error:
                    nota = 6
                elif long_pol1 - error <= longitud <= long_pol1 + error or long_pol2 - error <= longitud <= long_pol2 + error:
                    nota = 5
                else:
                    nota = 4
            elif len_conductores == 1:
                clasificacion1['Error p1'].append(0)
                clasificacion1['Error p2'].append(0)
                clasificacion1['Error p3'].append(0)
                x1, y1, z1 = get_coord(conductores[0]['VERTICES'])
                long_pol1 = np.sqrt((x1[0]-x1[-1])**2 + (y1[0]-y1[-1])**2)
                clasificacion1['Longitud polilínea 1'].append(long_pol1)
                clasificacion1['Longitud polilínea 2'].append(0)
                clasificacion1['Longitud polilínea 3'].append(0)
                error = 5
                if long_pol1 - error <= longitud <= long_pol1 + error:
                    nota = 3
                else:
                    nota = 2
            else:
                clasificacion1['Longitud polilínea 1'].append(0)
                clasificacion1['Longitud polilínea 2'].append(0)
                clasificacion1['Longitud polilínea 3'].append(0)
                clasificacion1['Error p1'].append(0)
                clasificacion1['Error p2'].append(0)
                clasificacion1['Error p3'].append(0)
                nota = 1
            clasificacion1['Puntuación'].append(nota)
        else:
            clasificacion1['Puntuación'].append(0)
            clasificacion1['Longitud polilínea 1'].append(0)
            clasificacion1['Longitud polilínea 2'].append(0)
            clasificacion1['Longitud polilínea 3'].append(0)
            clasificacion1['Error p1'].append(0)
            clasificacion1['Error p2'].append(0)
            clasificacion1['Error p3'].append(0)

    clasificacion1 = pd.DataFrame(clasificacion1)
    return(clasificacion1)

In [21]:
VDI = puntuación(pathdata0)
VDI

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
0,C_13411888_13411889,2,3,183.321200,182.523687,180.962356,182.198753,10
1,C_13412048_13412052,2,3,120.749507,120.645027,120.093174,120.058392,10
2,C_13412143_13412146,2,3,96.534936,95.711866,95.840392,95.421472,10
3,C_13412119_13412124,2,3,111.734681,111.710010,111.386610,110.893187,10
4,C_13412096_13412097,2,3,105.800523,105.092137,105.767811,104.833146,10
...,...,...,...,...,...,...,...,...
124,C_13411824_13411828,2,6,162.830114,100.930682,100.873231,100.851676,7
125,C_13411775_13411776,2,4,153.788057,47.436625,30.332399,33.117415,7
126,C_13434480_13434481,1,0,160.414987,0.000000,0.000000,0.000000,0
127,C_13434455_13434456,2,3,187.420354,186.426493,187.333075,186.787969,10


Según ellos estos vanos están incompletos

In [22]:
VDI[VDI['Vano']=='C_13434448_13434449']
VDI[VDI['Vano']=='C_13411888_13411891']
VDI[VDI['Vano']=='C_13411767_13411770'] # A este le pone un 10
# Según ellos a un conductor le falta un trozo en el extremo, puede ser los 5 cm del tercero que lo cuenta dentro del error
VDI[VDI['Vano']=='C_13411767_13411776']
VDI[VDI['Vano']=='C_13411775_13411776']
VDI[VDI['Vano']=='C_13411824_13411828']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
124,C_13411824_13411828,2,6,162.830114,100.930682,100.873231,100.851676,7


Según ellos estos son casos raros pero correctos: la forma en la que se une el conductor al apoyo es distinta en un apoyo y en el otro. Todos los clasificamos como 10.

In [23]:
VDI[VDI['Vano']=='C_13412064_13412065']
VDI[VDI['Vano']=='C_13412051_13412064']
VDI[VDI['Vano']=='C_13412143_13412144']
VDI[VDI['Vano']=='C_13412141_13412144']
VDI[VDI['Vano']=='C_13412101_13412104']
VDI[VDI['Vano']=='C_13412104_13412105']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
110,C_13412104_13412105,2,3,65.666767,65.441792,65.576043,64.360799,10


Casos con huecos y dos apoyos

In [24]:
VDI[(VDI['Puntuación']<10) & (VDI['Apoyos']==2)]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
7,C_13412137_13412138,2,4,113.842994,4.339028,111.464067,110.757673,9
77,C_13412084_13412085,2,0,121.784992,0.000000,0.000000,0.000000,1
81,C_13411824_13411826,2,4,12.382514,6.451877,10.293428,11.653459,9
92,C_13434448_13434449,2,3,110.388767,82.928874,80.664982,81.415168,7
104,C_13411888_13412152,2,4,175.425436,31.892372,174.419977,174.236204,9
106,C_13411767_13411776,2,3,153.247780,153.138323,152.024707,143.032621,9
111,C_13411888_13411891,2,9,111.354326,6.743740,2.841455,54.711682,7
124,C_13411824_13411828,2,6,162.830114,100.930682,100.873231,100.851676,7
125,C_13411775_13411776,2,4,153.788057,47.436625,30.332399,33.117415,7


Casos sin cables

In [25]:
VDI[VDI['Conductores']==0]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
55,C_13411667_13411668,1,0,99.956944,0.0,0.0,0.0,0
59,C_13434463_13434464,1,0,177.388953,0.0,0.0,0.0,0
77,C_13412084_13412085,2,0,121.784992,0.0,0.0,0.0,1
126,C_13434480_13434481,1,0,160.414987,0.0,0.0,0.0,0


Casos con un solo apoyo cartografiado

In [26]:
VDI[VDI['Apoyos']==1]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
29,C_13411890_13411891,1,3,113.996274,0.0,0.0,0.0,0
40,C_13411729_13411832,1,3,75.532068,0.0,0.0,0.0,0
41,C_13411725_13411730,1,3,124.383025,0.0,0.0,0.0,0
52,C_13411885_13411890,1,3,96.558853,0.0,0.0,0.0,0
55,C_13411667_13411668,1,0,99.956944,0.0,0.0,0.0,0
56,C_13411725_13411728,1,3,147.546812,0.0,0.0,0.0,0
59,C_13434463_13434464,1,0,177.388953,0.0,0.0,0.0,0
89,C_13411829_13411830,1,3,144.676772,0.0,0.0,0.0,0
112,C_13411772_13411889,1,3,117.171659,0.0,0.0,0.0,0
126,C_13434480_13434481,1,0,160.414987,0.0,0.0,0.0,0


In [27]:
REG = puntuación(pathdata1)

Casos que según ellos están incompletos, todos tienen puntuación menor a 10

In [28]:
REG[REG['Vano']=='G_13578357_13578466']
REG[REG['Vano']=='G_13578391_13578442']
REG[REG['Vano']=='G_13578431_13578442']
REG[REG['Vano']=='G_13578321_13578371']
REG[REG['Vano']=='G_13578400_13578433']
REG[REG['Vano']=='G_13578400_13578439']
REG[REG['Vano']=='G_13578364_13578407']
REG[REG['Vano']=='G_13578407_13578444']
REG[REG['Vano']=='G_13578334_13578477']
REG[REG['Vano']=='G_13578402_13578415']
REG[REG['Vano']=='G_13578285_13578315']
REG[REG['Vano']=='G_13578315_13578417']
REG[REG['Vano']=='G_13578396_13578425']
REG[REG['Vano']=='G_13578381_13578395']
REG[REG['Vano']=='G_13578381_13578405']
REG[REG['Vano']=='G_13578320_13578341']
REG[REG['Vano']=='G_13578307_13578514']
REG[REG['Vano']=='G_13576792_13578514']
REG[REG['Vano']=='G_13576834_13576897']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
85,G_13576834_13576897,2,8,28.421093,4.93308,4.939488,8.255626,7


Casos raros pero correctos según ellos, todos con 10

In [29]:
REG[REG['Vano']=='G_13563638_13578462']
REG[REG['Vano']=='G_13578306_13578466']
REG[REG['Vano']=='G_13578388_13578466']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
74,G_13578388_13578466,2,3,152.517673,150.945988,151.975202,149.69239,10


Casos con 6 conductores

In [30]:
REG[REG['Vano']=='G_13576786_13576866']
REG[REG['Vano']=='G_13576786_13576844']
REG[REG['Vano']=='G_13576775_13576858']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
140,G_13576775_13576858,2,6,140.120394,139.399496,138.856837,139.412017,10


Casos con huecos y dos apoyos

In [31]:
REG[(REG['Puntuación']<10) & (REG['Apoyos']==2)]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
5,G_13578307_13578514,2,3,110.920768,101.002281,101.761087,99.993435,7
19,G_13576769_13576858,2,9,20.233636,5.442803,7.152198,6.289819,7
23,G_13578374_13578463,2,4,93.352973,23.978445,92.564478,91.610084,9
28,G_13578407_13578444,2,3,114.914590,107.568255,107.384554,106.948923,7
31,G_13578311_13578407,2,6,121.417233,118.821322,9.559160,8.504681,8
33,G_13576740_13576792,2,7,187.972754,3.593759,187.551476,187.740847,9
37,G_13578381_13578405,2,5,8.909849,3.343597,4.140818,8.683749,9
40,G_13578400_13578433,2,3,72.386413,49.379958,70.812803,70.847190,9
44,G_13576792_13576854,2,3,12.990054,7.840184,7.818122,7.706452,7
46,G_13578334_13578477,2,3,82.878197,61.385290,61.653832,65.551940,7


Casos sin cables

In [32]:
REG[REG['Conductores']==0]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
24,G_13578285_13578315,1,0,40.191562,0.0,0.0,0.0,0
45,G_13578391_13578442,1,0,44.153244,0.0,0.0,0.0,0
121,G_13578341_13578423,1,0,27.785554,0.0,0.0,0.0,0
147,G_13578381_13578395,1,0,68.536556,0.0,0.0,0.0,0


Casos con un solo apoyo cartografiado

In [33]:
REG[REG['Apoyos']<2]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
13,G_13576748_13576812,1,12,95.761966,0.0,0.0,0.0,0
24,G_13578285_13578315,1,0,40.191562,0.0,0.0,0.0,0
45,G_13578391_13578442,1,0,44.153244,0.0,0.0,0.0,0
58,G_13528124_13528233,1,6,90.907411,0.0,0.0,0.0,0
121,G_13578341_13578423,1,0,27.785554,0.0,0.0,0.0,0
142,G_13576748_13576801,1,7,184.126400,0.0,0.0,0.0,0
147,G_13578381_13578395,1,0,68.536556,0.0,0.0,0.0,0


In [34]:
XIN = puntuación(pathdata2)
XIN

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
0,G_13645821_13645987,2,3,202.041257,201.990184,200.911766,201.967452,10
1,G_13645776_13645990,2,6,244.909157,118.807072,118.671604,119.206091,7
2,G_13645684_13645777,2,3,152.133393,150.896687,151.299593,152.077406,10
3,G_13615863_13645824,2,3,51.923733,51.567533,51.373664,51.831098,10
4,G_13645732_13645823,2,3,153.801037,153.675813,153.546489,153.286383,10
...,...,...,...,...,...,...,...,...
158,G_13645826_13645909,2,6,183.165321,7.215530,7.488864,6.745865,7
159,G_13645647_13645790,2,3,107.718973,107.567536,107.567070,107.501833,10
160,G_13645975_13645976,2,3,105.598647,104.556848,105.479120,105.229700,10
161,G_13645856_13645977,2,3,105.370797,104.541596,104.968118,104.325176,10


Casos incompletos según ellos, todos con puntuación menor a 10

In [35]:
XIN[XIN['Vano']=='G_13645875_13645996']
XIN[XIN['Vano']=='G_13037116_13511437']
XIN[XIN['Vano']=='G_13037116_13645737']
XIN[XIN['Vano']=='G_13645871_13645951']
XIN[XIN['Vano']=='G_13645681_13645990']
XIN[XIN['Vano']=='G_13645862_13645986']
XIN[XIN['Vano']=='G_13645821_13645862']
XIN[XIN['Vano']=='G_13645946_13645982']
XIN[XIN['Vano']=='G_13088932_13149467']
XIN[XIN['Vano']=='G_13645720_13645976']
XIN[XIN['Vano']=='G_13645666_13645811']
XIN[XIN['Vano']=='G_13088936_13145989']
XIN[XIN['Vano']=='G_13645800_13645927']
XIN[XIN['Vano']=='G_13645843_13645881']
XIN[XIN['Vano']=='G_13645790_13645962']
XIN[XIN['Vano']=='G_13645645_13645918']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
78,G_13645645_13645918,2,6,154.963127,4.663635,7.134875,9.075237,7


Casos raros según ellos

In [36]:
XIN[XIN['Vano']=='G_13618881_13645911']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
116,G_13618881_13645911,2,7,67.691565,7.448992,7.071461,7.997245,7


In [37]:
XIN[XIN['Vano']=='G_13618881_13645690']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
8,G_13618881_13645690,2,4,131.2791,2.030791,130.417053,130.706328,9


In [38]:
XIN[XIN['Vano']=='G_13645910_13645997']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
117,G_13645910_13645997,2,3,169.546231,169.419558,169.182754,169.379625,10


In [39]:
XIN[XIN['Vano']=='G_13645910_13645956']

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
36,G_13645910_13645956,2,3,184.573779,184.304745,183.961639,183.080011,10


Casos con huecos y dos apoyos

In [40]:
XIN[(XIN['Puntuación']<10) & (XIN['Apoyos']==2)]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
1,G_13645776_13645990,2,6,244.909157,118.807072,118.671604,119.206091,7
5,G_13499432_13645947,2,6,70.242467,8.742501,10.697377,68.848330,8
7,G_13037116_13511437,2,6,49.578426,4.587040,6.701495,6.925042,7
8,G_13618881_13645690,2,4,131.279100,2.030791,130.417053,130.706328,9
9,G_13645790_13645962,2,5,183.679737,2.667129,8.081915,156.015421,7
14,G_13645707_13645750,2,6,276.639144,144.864490,131.641360,131.369608,7
23,G_13645668_13645817,2,5,145.225637,143.597812,2.724006,143.945486,9
24,G_13645771_13645980,2,3,127.917143,116.797201,118.494385,126.795711,8
39,G_13645947_13645992,2,6,138.013584,8.722938,8.984318,11.817287,7
47,G_13645821_13645862,2,3,12.900240,7.230941,8.256513,6.651360,8


Casos sin cable

In [41]:
XIN[XIN['Conductores']==0]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
40,G_13645661_13645713,1,0,243.130898,0.0,0.0,0.0,0
61,G_13645712_13645761,1,0,117.083952,0.0,0.0,0.0,0


Casos con un solo apoyo

In [42]:
XIN[XIN['Apoyos']<2]

,Vano,Apoyos,Conductores,Longitud 2D,Longitud polilínea 1,Longitud polilínea 2,Longitud polilínea 3,Puntuación
40,G_13645661_13645713,1,0,243.130898,0.0,0.0,0.0,0
61,G_13645712_13645761,1,0,117.083952,0.0,0.0,0.0,0
100,G_13645739_13645998,1,3,143.339226,0.0,0.0,0.0,0


: 